In [2]:
import numpy as np
import pandas as pd
from statsbombpy import sb
#pd.set_option('display.max_rows', None)
from mplsoccer import Pitch, Sbopen
from mplsoccer import VerticalPitch,Pitch
import networkx as nx
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx



In [21]:
df_competitions.head(30)


,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2023-08-17T23:51:11.837478,None,None,2023-08-17T23:51:11.837478
1,16,4,Europe,Champions League,male,False,False,2018/2019,2023-03-07T12:20:48.118250,2021-06-13T16:17:31.694,None,2023-03-07T12:20:48.118250
2,16,1,Europe,Champions League,male,False,False,2017/2018,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2021-01-23T21:55:30.425330
3,16,2,Europe,Champions League,male,False,False,2016/2017,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
4,16,27,Europe,Champions League,male,False,False,2015/2016,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
5,16,26,Europe,Champions League,male,False,False,2014/2015,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
6,16,25,Europe,Champions League,male,False,False,2013/2014,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
7,16,24,Europe,Champions League,male,False,False,2012/2013,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2021-07-10T13:41:45.751
8,16,23,Europe,Champions League,male,False,False,2011/2012,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
9,16,22,Europe,Champions League,male,False,False,2010/2011,2022-01-26T21:07:11.033473,2021-06-13T16:17:31.694,None,2022-01-26T21:07:11.033473


In [22]:
parser = Sbopen()
df_competitions = parser.competition()
df_competitions.head(30)
df = parser.match(competition_id=9, season_id=27)
df.head()

,match_id,match_date,kick_off,home_score,away_score,match_status,match_status_360,last_updated,last_updated_360,match_week,...,competition_stage_id,competition_stage_name,stadium_id,stadium_name,stadium_country_id,stadium_country_name,referee_id,referee_name,referee_country_id,referee_country_name
0,3890561,2016-05-14,2016-05-14 15:30:00,1,4,available,unscheduled,2023-07-24 07:47:12.510532,NaT,34,...,1,Regular Season,4541,PreZero Arena,85,Germany,186,Felix Brych,85,Germany
1,3890505,2016-04-02,2016-04-02 15:30:00,1,0,available,unscheduled,2023-08-03 13:14:14.503553,NaT,28,...,1,Regular Season,4867,Allianz Arena,85,Germany,1008758,Florian Meyer,85,Germany
2,3890511,2016-04-08,2016-04-08 20:30:00,2,2,available,unscheduled,2023-08-06 05:50:38.345688,NaT,29,...,1,Regular Season,367,Olympiastadion Berlin,85,Germany,235,Benjamin Brand,85,Germany
3,3890515,2016-04-09,2016-04-09 15:30:00,1,2,available,unscheduled,2023-07-18 17:52:17.363623,NaT,29,...,1,Regular Season,379,Volksparkstadion,85,Germany,1008767,Peter Sippel,85,Germany
4,3890411,2015-12-20,2015-12-20 16:30:00,2,0,available,unscheduled,2023-07-16 13:47:06.565811,NaT,17,...,1,Regular Season,367,Olympiastadion Berlin,85,Germany,1008767,Peter Sippel,85,Germany


In [23]:
first_row = df.iloc[0]

# Extract 'away_team_name' and 'home_team_name' values from the first row
away_team = first_row['away_team_name']
home_team = first_row['home_team_name']

# Print the values
print("Away Team:", away_team)
print("Home Team:", home_team)


Away Team: Schalke 04
Home Team: Hoffenheim


In [24]:
df, related, freeze, tactics = parser.event(3890561)
sub = df.loc[df["type_name"] == "Substitution"].loc[df["team_name"] == "Hoffenheim"].iloc[0]["index"]
mask_uruguay = (df.type_name == 'Pass') & (df.team_name == "Hoffenheim") & (df.index < sub) & (df.outcome_name.isnull()) & (df.sub_type_name != "Throw-in")
df_pass = df.loc[mask_uruguay, ['x', 'y', 'end_x', 'end_y', "player_name", "pass_recipient_name"]]
df_pass["player_name"] = df_pass["player_name"].apply(lambda x: str(x).split()[-1])
df_pass["pass_recipient_name"] = df_pass["pass_recipient_name"].apply(lambda x: str(x).split()[-1])


In [ ]:
df_pass

In [ ]:
scatter_df = pd.DataFrame()
for i, name in enumerate(df_pass["player_name"].unique()):
    passx = df_pass.loc[df_pass["player_name"] == name]["x"].to_numpy()
    recx = df_pass.loc[df_pass["pass_recipient_name"] == name]["end_x"].to_numpy()
    passy = df_pass.loc[df_pass["player_name"] == name]["y"].to_numpy()
    recy = df_pass.loc[df_pass["pass_recipient_name"] == name]["end_y"].to_numpy()
    scatter_df.at[i, "player_name"] = name
    #make sure that x and y location for each circle representing the player is the average of passes and receptions
    scatter_df.at[i, "x"] = np.mean(np.concatenate([passx, recx]))
    scatter_df.at[i, "y"] = np.mean(np.concatenate([passy, recy]))
    #calculate number of passes
    scatter_df.at[i, "no"] = df_pass.loc[df_pass["player_name"] == name].count().iloc[0]

#adjust the size of a circle so that the player who made more passes
scatter_df['marker_size'] = (scatter_df['no'] / scatter_df['no'].max() * 1500)

In [7]:
df_pass["pair_key"] = df_pass.apply(lambda x: "_".join(sorted([x["player_name"], x["pass_recipient_name"]])), axis=1)
lines_df = df_pass.groupby(["pair_key"]).x.count().reset_index()
lines_df.rename({'x':'pass_count'}, axis='columns', inplace=True)
lines_df = lines_df[lines_df['pass_count']>2]

In [ ]:
df_pass

In [ ]:
#Drawing pitch
pitch = Pitch(line_color='grey')
fig, ax = pitch.grid(grid_height=0.9, title_height=0.06, axis=False,
                     endnote_height=0.04, title_space=0, endnote_space=0)
#Scatter the location on the pitch
pitch.scatter(scatter_df.x, scatter_df.y, s=scatter_df.marker_size, color='skyblue', edgecolors='grey', linewidth=1, alpha=1, ax=ax["pitch"], zorder = 3)
#annotating player name
for i, row in scatter_df.iterrows():
    pitch.annotate(row.player_name, xy=(row.x, row.y), c='black', va='center', ha='center', weight = "bold", size=16, ax=ax["pitch"], zorder = 4)

fig.suptitle("Nodes location - Uruguay", fontsize = 30)
plt.show()

In [ ]:
pitch = Pitch(line_color='grey')
fig, ax = pitch.grid(grid_height=0.9, title_height=0.06, axis=False,
                     endnote_height=0.04, title_space=0, endnote_space=0)
pitch.scatter(scatter_df.x, scatter_df.y, s=scatter_df.marker_size, color='skyblue', edgecolors='grey', linewidth=1, alpha=1, ax=ax["pitch"], zorder = 3)
for i, row in scatter_df.iterrows():
    pitch.annotate(row.player_name, xy=(row.x, row.y), c='black', va='center', ha='center', weight = "bold", size=16, ax=ax["pitch"], zorder = 4)

for i, row in lines_df.iterrows():
        player1 = row["pair_key"].split("_")[0]
        player2 = row['pair_key'].split("_")[1]
        #take the average location of players to plot a line between them
        player1_x = scatter_df.loc[scatter_df["player_name"] == player1]['x'].iloc[0]
        player1_y = scatter_df.loc[scatter_df["player_name"] == player1]['y'].iloc[0]
        player2_x = scatter_df.loc[scatter_df["player_name"] == player2]['x'].iloc[0]
        player2_y = scatter_df.loc[scatter_df["player_name"] == player2]['y'].iloc[0]
        num_passes = row["pass_count"]
        #adjust the line width so that the more passes, the wider the line
        line_width = (num_passes / lines_df['pass_count'].max() * 10)
        #plot lines on the pitch
        pitch.lines(player1_x, player1_y, player2_x, player2_y,
                        alpha=1, lw=line_width, zorder=2, color="skyblue", ax = ax["pitch"])

fig.suptitle("Uruguay Passing Network against South Korea", fontsize = 30)
plt.show()

In [25]:
def digraph(data):
        G = nx.DiGraph()
        pass_counts = {}
        players = data["player_name"].unique()
        G.add_nodes_from(players)
        for _, row in data.iterrows():
                key = (row["player_name"], row["pass_recipient_name"])
                if key in pass_counts:
                        pass_counts[key] += 1
                else:
                        pass_counts[key] = 1

        for key, weight in pass_counts.items():
                G.add_edge(key[0], key[1], weight=weight)
        
        return G
G = digraph(df_pass)
G.edges

OutEdgeView([('Uth', 'Kramarić'), ('Uth', 'Schwegler'), ('Uth', 'Kadeřábek'), ('Kramarić', 'Rudy'), ('Kramarić', 'Kadeřábek'), ('Kramarić', 'Elyounoussi'), ('Kramarić', 'Schär'), ('Kramarić', 'Strobl'), ('Kramarić', 'Bičakčić'), ('Kramarić', 'Uth'), ('Kramarić', 'Volland'), ('Rudy', 'Schär'), ('Rudy', 'Strobl'), ('Rudy', 'Volland'), ('Rudy', 'Kramarić'), ('Rudy', 'Elyounoussi'), ('Rudy', 'Uth'), ('Rudy', 'Schwegler'), ('Schär', 'Strobl'), ('Schär', 'Uth'), ('Schär', 'Bičakčić'), ('Schär', 'Volland'), ('Schär', 'Elyounoussi'), ('Schär', 'Schwegler'), ('Schär', 'Rudy'), ('Schär', 'Grahl'), ('Schär', 'Kadeřábek'), ('Schär', 'Kramarić'), ('Strobl', 'Bičakčić'), ('Strobl', 'Elyounoussi'), ('Strobl', 'Rudy'), ('Strobl', 'Schär'), ('Strobl', 'Schwegler'), ('Strobl', 'Kramarić'), ('Strobl', 'Volland'), ('Strobl', 'Kadeřábek'), ('Bičakčić', 'Strobl'), ('Bičakčić', 'Uth'), ('Bičakčić', 'Rudy'), ('Bičakčić', 'Schär'), ('Bičakčić', 'Kadeřábek'), ('Bičakčić', 'Kramarić'), ('Bičakčić', 'Schwegler'),

In [ ]:

def draw_graph(G):
    layout = nx.spring_layout(G)
    nx.draw(G, layout, with_labels=True, node_size=800, node_color='skyblue')
    labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, layout, edge_labels=labels)
    edge_widths = [weight for _, _, weight in G.edges(data='weight')]
    nx.draw(G, layout, edgelist=G.edges(), width=edge_widths, edge_color='lightblue', edge_cmap=plt.cm.Blues, edge_vmin=min(edge_widths), edge_vmax=max(edge_widths), alpha=0.9)
    plt.show()
draw_graph(G)


In [ ]:
def digraph(data):
        G = nx.DiGraph()
        pass_counts = {}
        players = data["player_name"].unique()
        G.add_nodes_from(players)
        for _, row in data.iterrows():
                key = (row["player_name"], row["pass_recipient_name"])
                if key in pass_counts:
                        pass_counts[key] += 1
                else:
                        pass_counts[key] = 1

        for key, weight in pass_counts.items():
                G.add_edge(key[0], key[1], weight=weight)
        
        return G
def match_list(wc):
    matches = list()
    for i in wc['match_id']:
        matches.append(i)
    return matches

data_mf = pd.DataFrame()

In [9]:
G = digraph(df_pass)


In [26]:
def avg(dictionary):
    total = sum(dictionary.values())

    average = total / len(dictionary)
    
    return average
avg(a)

0.05555555555555556

In [17]:
nx.eigenvector_centrality(G)
nx.pagerank(G)

{'Mendy': 0.04749137756782092,
 'Romao': 0.1534830974701256,
 'Mandanda': 0.03861271304518332,
 'Rekik': 0.07291374426898974,
 'Thauvin': 0.10971140478966913,
 'Cabella': 0.17650788016799185,
 'Isla': 0.11165859970187089,
 'Mbida': 0.08832504490828376,
 'Tunga': 0.06042593332203804,
 'Gaitán': 0.08129498372575866,
 'Ndoubena': 0.05957522103226822}

In [27]:
nx.pagerank(G)

{'Uth': 0.06308639834579978,
 'Kramarić': 0.0909384783157926,
 'Rudy': 0.07944652494497412,
 'Schär': 0.13054359735643012,
 'Strobl': 0.1092215016857459,
 'Bičakčić': 0.10272844210955924,
 'Elyounoussi': 0.1232416402119067,
 'Schwegler': 0.10460531765456703,
 'Kadeřábek': 0.0774927778925357,
 'Volland': 0.08250919522123826,
 'Grahl': 0.03618612626145051}